## 模型融合

这里使用了相对简单的融合方法，对模型做一个blending，相比之stacking，在数据比赛中，通常blending用得更多，相对而言没有那么容易overfitting。

In [ ]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
import h5py
import os

submission = pd.read_csv('../input/sample_submission.csv')

# read in RF results
with h5py.File('../output/probs/allpreds.h5', 'r') as hf:
        predshf = hf['preds_latest']
        preds = 0.31*normalize(predshf.value, norm='l1', axis=1)

# read in XGB results
with h5py.File('../output/probs/allpreds_xgb.h5', 'r') as hf:
        predshf = hf['preds']
        preds += 0.39*normalize(predshf.value, norm='l1', axis=1)

# read in SGD results
with h5py.File('../output/probs/allpreds_sgd.h5', 'r') as hf:
        predshf = hf['preds']
        preds += 0.27*normalize(predshf.value, norm='l1', axis=1)

# read in Bernoulli NB results
with h5py.File('../output/probs/allpreds_bnb.h5', 'r') as hf:
        predshf = hf['preds']
        preds += 0.03*normalize(predshf.value, norm='l1', axis=1)

print('generating submission')
col_ind = np.argsort(-preds, axis=1)[:,:5]
hc = [' '.join(row.astype(str)) for row in col_ind]

sub = pd.DataFrame(data=hc, index=submission.id)
sub.reset_index(inplace=True)
sub.columns = submission.columns
sub.to_csv('../output/pred_sub.csv', index=False)